In [1]:
import pandas as pd 
import numpy as np
import sklearn
import os
import collections
from sklearn import metrics,model_selection,linear_model
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LinearRegression,BayesianRidge,LogisticRegression
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
def transform(x):
    if x=='Unknown':
        return 'Dialogue level'
    else:
        return x
    
def assign_tone(row):
    if row['backchannels'] > 0 or row['code-switching for communicative purposes'] > 0 or row['collaborative finishes'] > 0:
        return 'Informal'
    elif row['subordinate clauses'] > 0 or row['impersonal subject + non-factive verb + NP'] > 0:
        return 'Formal'
    else:
        return 'Neutral'  # Default to Neutral if none of the criteria match
    
    
def get_new_labels(counts_df,new_column='P-V',old_column=['topic extension with the same content',
                                                          'topic extension under the previous direction']):
    cc_dd=counts_df[old_column].astype(int).values
    cc_dd=['-'.join([str(j) for j in i]) for i in cc_dd]
    counts_df[new_column]=cc_dd
    return counts_df

def get_result(test_y,test_pred):
    acc=metrics.accuracy_score(test_y,test_pred)
    pre=metrics.precision_score(test_y,test_pred)
    rec=metrics.recall_score(test_y,test_pred)
    f1s=metrics.f1_score(test_y,test_pred)
    return [acc,pre,rec,f1s]

def get_result(test_y,test_pred):
    acc=metrics.accuracy_score(test_y,test_pred)
    pre=metrics.precision_score(test_y,test_pred,average='weighted')
    rec=metrics.recall_score(test_y,test_pred,average='weighted')
    f1s=metrics.f1_score(test_y,test_pred,average='weighted')
    return [acc,pre,rec,f1s]

def get_result_2(test_y,test_pred):
    mae=metrics.mean_absolute_error(test_y,test_pred)
    mse=metrics.mean_squared_error(test_y,test_pred)
    mape=metrics.mean_absolute_percentage_error(test_y,test_pred)
    r2=metrics.r2_score(test_y,test_pred)
    return [mae,mse,mape,r2]

def get_normal(data):
    #============归一化===============
    normal_list=[]
    for i in data.values:
        if i[2]=='Token level':
            norm=len(i[3].split('&&&&')[1].split())/Token_level_label_count[i[1]]
        elif i[2]=='Utterance level':
            norm=len(i[3].split('&&&&')[1].split())/Utterance_level_label_count[i[1]]
        else:
            norm=-1
        normal_list.append(norm)

    data['normal']=normal_list
    return data


score_dict={'topic extension with clear new context':[5,'Y1'],
           'topic extension under the previous direction':[4,'Y1'],
           'topic extension with the same content':[3,'Y1'],
            'repeat and no topic extension':[2,'Y1'],
            'no topic extension and stop the topic at this point':[1,'Y1'],
            
            'overall tone choice: very informal':[5,'Y2'],
           'overall tone choice: quite informal, but some expressions are still formal':[4,'Y2'],
           'overall tone choice: quite formal and some expressions are not that formal':[3,'Y2'],
            'overall tone choice: quite formal and some expressions are not that formal':[2,'Y2'],
            'overall tone choice: very formal':[1,'Y2'],
            
            'Co1':[5,'Y3'],
           'Co2':[4,'Y3'],
           'Co3':[3,'Y3'],
            'Co4':[2,'Y3'],
            'Co5':[1,'Y3'],
            
           'Cc1':[5,'Y4'],
           'Cc2':[4,'Y4'],
           'Cc3':[3,'Y4'],
            'Cc4':[2,'Y4'],
            'Cc5':[1,'Y4'],
            
            'conversation opening':[3,'Y3'],
            'onversation closing':[3,'Y4']
           }

In [4]:
data_path='./data_csv_sample/'
data=pd.DataFrame({})
for file in os.listdir(data_path):
    file_path=data_path+file
    data_i=pd.read_csv(file_path)
    data=pd.concat([data,data_i])
    
    
new_labellevel=[]
for i in data.values:
    _,i_label,i_labellevel,_=i
    if i_labellevel=='Unknown':
        if i_label=='adjectives/ adverbs expressing possibility':
            new_v='Utterance level'
        else:
            new_v='Dialogue level'
    else:
        new_v=i_labellevel
        
    new_labellevel.append(new_v)
    
data['LabelLevel']=new_labellevel
# data['LabelLevel']=data['LabelLevel'].apply(transform)
data.index=range(len(data))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 1096: invalid start byte

In [ ]:
#=============统计频次============
Token_level=data[data['LabelLevel']=='Token level']
Token_level_label_count=collections.Counter(Token_level['Label'])

Utterance_level=data[data['LabelLevel']=='Utterance level']
Utterance_level_label_count=collections.Counter(Utterance_level['Label'])

Dialogue_level=data[data['LabelLevel']=='Dialogue level']
Dialogue_level_label_count=collections.Counter(Dialogue_level['Label'])

print(len(Utterance_level_label_count)+len(Token_level_label_count))
valid_keys_list=list(Utterance_level_label_count.keys())+list(Token_level_label_count.keys())

In [ ]:
all_people_df=pd.DataFrame({})
all_labels_df=pd.DataFrame({})
all_classs_df=pd.DataFrame({})
for index in range(0,len(data),10):
    index_data=data[index:index+10]
    index_data['word_num']=[len(i.split('&&&&')[1].split()) for i in index_data['Content']]#统计各句子的特殊符号之间的word数
    
    #===========================================================提取feature=================
    #===========统计每一个Label对应的平均词汇数============
    test_feature={}
    for label in index_data['Label'].unique():
        index_data_label=index_data[index_data['Label']==label]
        index_data_label_1=pd.DataFrame([collections.Counter(index_data_label['word_num'])],index=[label])/len(index_data_label)
        test_feature[label]=sum(np.array(index_data_label_1.columns)*index_data_label_1.values[0])/(index_data_label_1.sum().sum())
    test_feature=pd.DataFrame([test_feature],index=[index])
    all_people_df=pd.concat([all_people_df,test_feature])#===========特征矩阵
    
    #===========================================================提取score=================
    #===========查表获取分数===========
    new_score_list=[]
    for label in index_data['Label']:
        if label in score_dict:
            new_score=score_dict[label]
        else:
            new_score=[None,None]
        new_score_list.append(new_score)
    index_data[['score','type']]=new_score_list

    #============求取平均分数=============
    type_score={}
    type_class={}
    for type_ in ['Y1','Y2','Y3','Y4']:
        type_data=index_data[index_data['type']==type_]
        if len(type_data)==0:
            type_score[type_]=3
        else:
            type_score[type_]=index_data['score'].mean()
            
        type_data_big3=type_data[type_data['score']>3]
        type_data_sma3=type_data[type_data['score']<3]
        if len(type_data_big3)+len(type_data_sma3)==0:
            type_class[type_]=3
        else:
            if len(type_data_sma3)>0:
                type_data_sma3=type_data_sma3.sort_values(by=['score','word_num'])
                type_data_sma3_s=type_data_sma3.values[0,-2]
                type_data_sma3=type_data_sma3[type_data_sma3['score']==type_data_sma3_s]
                type_data_sma3_w=type_data_sma3.values[-1,-3]
                type_data_sma3=type_data_sma3[type_data_sma3['word_num']==type_data_sma3_w]
            if len(type_data_big3)>0:
                type_data_big3=type_data_big3.sort_values(by=['score','word_num'])
                type_data_big3_s=type_data_big3.values[-1,-2]
                type_data_big3=type_data_big3[type_data_big3['score']==type_data_big3_s]
                type_data_big3_w=type_data_big3.values[-1,-3]
                type_data_big3=type_data_big3[type_data_big3['word_num']==type_data_big3_w]
            type_data_concat=pd.concat([type_data_sma3,type_data_big3])
            type_data_concat=type_data_concat.sort_values(by=['word_num'])
            type_data_concat_s=int(type_data_concat.values[-1,-2])
            type_class[type_]=type_data_concat_s


    type_score_df=pd.DataFrame([type_score],index=[index])
    all_labels_df=pd.concat([all_labels_df,type_score_df])#===========分数矩阵
    
    
    
    type_class_df=pd.DataFrame([type_class],index=[index])
    all_classs_df=pd.concat([all_classs_df,type_class_df])#===========分数矩阵

data_x=all_people_df[valid_keys_list]
data_x=data_x.fillna(0)
data_y=all_labels_df.copy()

columns_list=[]
for i in range(1,5):
    for j in range(1,6):
        columns_list.append(f'Y{i}_{j}')

data_class=all_classs_df.copy()

In [ ]:
train_x,test_x,train_y,test_y,train_c,test_c=train_test_split(data_x,data_y,data_class,train_size=0.8)
train_x_normal=(train_x-train_x.mean())/train_x.std()
test_x_normal=(test_x-train_x.mean())/train_x.std()

In [ ]:
train_x_normal.isna().sum().sum(),test_x_normal.isna().sum().sum()

In [8]:
topk=10

In [9]:
def save_csv(data,path):
    data.to_csv(f'./{path}.csv',encoding='utf_8_sig')

# Task1

In [10]:
model_dict={'Linear':LinearRegression(),'Logit':LogisticRegression(),
           'RF_R':RandomForestRegressor(),'RF_C':RandomForestClassifier(),
           'NB_R':BayesianRidge(),'NB_C':BernoulliNB()}

In [11]:
def get_all_result(train_x_normal,train_y,test_x_normal,test_y,model_name):
    global model
    test_pred_dict={}
    test_true_dict={}
    train_pred_dict={}
    train_true_dict={}
    train_result_linear={}
    test_result_linear={}
    model_infortance=pd.DataFrame({})
    for co in train_y.columns:
        train_true=train_y[co]
        test_true=test_y[co]
        model=model_dict[model_name]
        model.fit(train_x_normal,train_true)#训练

        train_pred=model.predict(train_x_normal)#预测
        test_pred=model.predict(test_x_normal)

        #整理
        if len(train_pred)==1:
            train_pred_dict[co]=train_pred
            test_pred_dict[co]=test_pred
        else:
            train_pred_dict[co]=train_pred[0]
            test_pred_dict[co]=test_pred[0]
        train_true_dict[co]=train_true
        test_true_dict[co]=test_true

        if model_name in ['Linear','NB_R','RF_R']:
            result_func=get_result_2
            metrics_list=['mae','mse','mape','r2']
        else:
            result_func=get_result
            metrics_list=['acc','pre','rec','f1s']
        train_result_linear[co]=result_func(train_true,train_pred)#评估
        test_result_linear[co]=result_func(test_true,test_pred)

        #--------得到重要度---------
        importance=None
        if model_name in ['Linear','NB_R']:
            importance=pd.DataFrame(model.coef_,index=model.feature_names_in_,columns=[co])
        elif model_name in ['RF_C','RF_R']:
            importance=pd.DataFrame(model.feature_importances_,index=model.feature_names_in_,columns=[co])
        elif model_name in ['Logit','NB_C']:
            importance=pd.DataFrame(model.coef_[0],index=model.feature_names_in_,columns=[co])
        
        importance_=abs(importance)
        importance_=importance_.sort_values(by=[co])
        importance=importance.loc[importance_[-topk:].index]
        model_infortance=pd.concat([model_infortance,importance],axis=1)

    train_result=pd.DataFrame(train_result_linear,index=metrics_list)
    test_result=pd.DataFrame(test_result_linear,index=metrics_list)
    test_pred_df=pd.DataFrame(test_pred_dict,index=test_y.index)
    test_true_df=pd.DataFrame(test_true_dict,index=test_y.index)
    train_pred_df=pd.DataFrame(train_pred_dict,index=train_y.index)
    train_true_df=pd.DataFrame(train_true_dict,index=train_y.index)
    return [train_result,test_result],[test_true_df,test_pred_df],[train_true_df,train_pred_df],model_infortance

In [12]:
#================Linear===========
model_name='Linear'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_y,
                                                                test_x_normal,test_y,
                                                                model_name)
importance,result

(                                                  Y1        Y2        Y3  \
 reference word                              0.044963       NaN  0.017658   
 epistemic copulas                          -0.051234  0.065111  0.012456   
 negotiation of meaning                      0.055360 -0.049189  0.020479   
 adjectives/ adverbs expressing possibility  0.057334       NaN  0.019268   
 routinized resources                        0.063352 -0.121988  0.012298   
 epistemic modals                            0.064683       NaN -0.018232   
 question-based responses                    0.072447       NaN       NaN   
 impersonal subject + non-factive verb + NP  0.107434 -0.040673       NaN   
 collaborative finishes                     -0.135893 -0.096509 -0.014633   
 tense choice to indicate interactive aims   0.221198  0.074357       NaN   
 non-factive verb phrase structure                NaN -0.036543       NaN   
 noun & verb collocation in proper form           NaN -0.046769       NaN   

In [13]:
#=================RF===============
model_name='RF_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_y,
                                                                test_x_normal,test_y,
                                                                model_name)
importance,result[1]

(                                                  Y1        Y2        Y3  \
 subordinate clauses                         0.049660  0.077255  0.094277   
 negotiation of meaning                      0.058192  0.080621  0.055193   
 epistemic modals                            0.059156       NaN       NaN   
 adjectives/ adverbs expressing possibility  0.062888  0.131648       NaN   
 feedback in the next turn                   0.071252  0.130809  0.133878   
 question-based responses                    0.080138       NaN  0.079582   
 noun & verb collocation in proper form      0.089041  0.046603  0.059381   
 impersonal subject + non-factive verb + NP  0.097746       NaN       NaN   
 reference word                              0.103224  0.053728       NaN   
 tense choice to indicate interactive aims   0.149270  0.086987       NaN   
 backchannels                                     NaN  0.069469  0.048236   
 formulaic responses                              NaN  0.072912  0.076170   

In [14]:
#=================RF===============
model_name='NB_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_y,
                                                                test_x_normal,test_y,
                                                                model_name)
importance,result[1]

(                                                  Y1        Y2        Y3  \
 routinized resources                        0.025668 -0.018018       NaN   
 negotiation of meaning                      0.026483 -0.012596  0.000272   
 reference word                              0.030029       NaN  0.000249   
 epistemic modals                            0.033496       NaN -0.000225   
 question-based responses                    0.034166       NaN       NaN   
 adjectives/ adverbs expressing possibility  0.035766       NaN  0.000273   
 backchannels                               -0.036000       NaN -0.000260   
 impersonal subject + non-factive verb + NP  0.052081 -0.008705       NaN   
 collaborative finishes                     -0.053656 -0.012550 -0.000283   
 tense choice to indicate interactive aims   0.098444  0.011136       NaN   
 formulaic responses                              NaN  0.005335       NaN   
 noun & verb collocation in proper form           NaN -0.007366       NaN   

In [15]:
#===============BP网络=============
def get_model_R(input_dim,output_dim):
    inputs=tf.keras.layers.Input(shape=(input_dim,))
    dense=tf.keras.layers.Dense(64,activation='relu')(inputs)
    dense=tf.keras.layers.Dense(64,activation='relu')(dense)
    outputs=tf.keras.layers.Dense(output_dim,activation='sigmoid')(dense)

    model=tf.keras.models.Model(inputs,outputs)
    model.compile(optimizer='adam',loss='mse')
    return model

def get_model_C(input_dim,output_dim):
    inputs=tf.keras.layers.Input(shape=(input_dim,))
    dense=tf.keras.layers.Dense(64,activation='relu')(inputs)
    dense=tf.keras.layers.Dense(64,activation='relu')(dense)
    outputs=tf.keras.layers.Dense(5,activation='softmax')(dense)

    model=tf.keras.models.Model(inputs,outputs)
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')
    return model

In [16]:
def get_BP_result(train_x_normal,train_y,test_x_normal,test_y,model_name):
    input_dim=train_x_normal.shape[1]
    output_dim=train_y.shape[1]

    if model_name=='BP_R':
        train_y_normal=train_y/5
        model_bp=get_model_R(input_dim,output_dim)
    else:
        train_y_normal=train_y.copy()
        model_bp=get_model_C(input_dim,output_dim)

    stopearly=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True,monitor='val_loss')
    history=model_bp.fit(train_x_normal,train_y_normal,
                         validation_split=0.1,batch_size=16,
                         epochs=100,callbacks=[stopearly],verbose=0)

    test_pred_bp=model_bp.predict(test_x_normal)
    train_pred_bp=model_bp.predict(train_x_normal)

    if model_name=='BP_R':
        test_pred_bp=test_pred_bp*5
        train_pred_bp=train_pred_bp*5
    else:
        train_pred_bp=(train_pred_bp>=0.5).astype(int)
        test_pred_bp=(test_pred_bp>=0.5).astype(int)

    test_pred_bp=pd.DataFrame(test_pred_bp,index=test_y.index,columns=test_y.columns)
    test_true_bp=test_y.copy()
    train_pred_bp=pd.DataFrame(train_pred_bp,index=train_y.index,columns=train_y.columns)
    train_true_bp=train_y.copy()

    train_result_dict={}
    test_result_dict={}

    if model_name in ['BP_R']:
        result_func=get_result_2
        metrics_list=['mae','mse','mape','r2']
    else:
        result_func=get_result
        metrics_list=['acc','pre','rec','f1s']

    for co in train_pred_bp.columns:
        true_train=train_true_bp[co]
        pred_train=train_pred_bp[co]
        true_test=test_true_bp[co]
        pred_test=test_pred_bp[co]

        result_train=result_func(true_train,pred_train)
        result_test=result_func(true_test,pred_test)
        train_result_dict[co]=result_train
        test_result_dict[co]=result_test

    train_result=pd.DataFrame(train_result_dict,index=metrics_list)
    test_result=pd.DataFrame(test_result_dict,index=metrics_list)
    model_infortance=None
    return [train_result,test_result],[test_true_bp,test_pred_bp],[train_true_bp,train_pred_bp],model_infortance

In [17]:
model_name='BP_R'
result,test_true_pred,train_true_pred,importance=get_BP_result(train_x_normal,train_y,
                                                               test_x_normal,test_y,model_name)
result

5/5 [==============================] - 0s 928us/step


[            Y1        Y2        Y3        Y4
 mae   0.455117  0.339253  0.184522  0.170859
 mse   0.425661  0.325621  0.108510  0.088346
 mape  0.155459  0.148770  0.070822  0.065814
 r2    0.071672  0.202801 -0.106851 -0.516190,
             Y1        Y2         Y3        Y4
 mae   0.435810  0.434854   0.197929  0.177816
 mse   0.421643  0.400226   0.073931  0.053570
 mape  0.148181  0.155865   0.064977  0.059272
 r2   -0.436197 -0.670321 -17.956649  0.000000]

# Task2

In [46]:
task_2_importanc={}
task_2_test_true_pred={}
task_2_train_true_pred={}

In [23]:
def get_result(test_y,test_pred):
    acc=metrics.accuracy_score(test_y,test_pred)
    pre=metrics.precision_score(test_y,test_pred,average='weighted')
    rec=metrics.recall_score(test_y,test_pred,average='weighted')
    f1s=metrics.f1_score(test_y,test_pred,average='weighted')
    return [acc,pre,rec,f1s]

In [47]:
model_name='Logit'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_c,
                                                                test_x_normal,test_c,
                                                                model_name)
task_2_importanc[model_name]=importance
task_2_test_true_pred[model_name]=test_true_pred
task_2_train_true_pred[model_name]=train_true_pred
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 

(                                                  Y1        Y2        Y3  \
 subordinate clauses                         0.116531 -0.396754 -0.377109   
 non-factive verb phrase structure          -0.151617  0.365288       NaN   
 collaborative finishes                      0.186891       NaN  0.574441   
 feedback in the next turn                   0.196870  0.405557       NaN   
 epistemic copulas                           0.226716 -0.632121  0.302988   
 question-based responses                   -0.260254       NaN       NaN   
 negotiation of meaning                     -0.431024  0.341880  0.237107   
 formulaic responses                        -0.441459       NaN       NaN   
 reference word                             -0.481514  0.520446       NaN   
 routinized resources                       -0.804706  0.788647       NaN   
 impersonal subject + non-factive verb + NP       NaN  0.296403       NaN   
 adjectives/ adverbs expressing possibility       NaN  0.529010 -0.508377   

In [49]:
model_name='RF_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_c,
                                                                test_x_normal,test_c,
                                                                model_name)
task_2_importanc[model_name]=importance
task_2_test_true_pred[model_name]=test_true_pred
task_2_train_true_pred[model_name]=train_true_pred
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 

(                                                  Y1        Y2        Y3  \
 formulaic responses                         0.061883  0.071716       NaN   
 negotiation of meaning                      0.063414  0.080021  0.054175   
 question-based responses                    0.064342       NaN  0.074857   
 subordinate clauses                         0.071053  0.089081  0.088212   
 backchannels                                0.073486  0.083901  0.095126   
 feedback in the next turn                   0.077736  0.068311  0.095859   
 adjectives/ adverbs expressing possibility  0.086330  0.078509       NaN   
 reference word                              0.092233  0.085290  0.093487   
 tense choice to indicate interactive aims   0.096827  0.080184       NaN   
 noun & verb collocation in proper form      0.111679  0.091853  0.091099   
 routinized resources                             NaN  0.079911  0.129442   
 epistemic copulas                                NaN       NaN  0.044587   

In [50]:
model_name='NB_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal,train_c,
                                                                test_x_normal,test_c,
                                                                model_name)
task_2_importanc[model_name]=importance
task_2_test_true_pred[model_name]=test_true_pred
task_2_train_true_pred[model_name]=train_true_pred
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

(                                                  Y1        Y2        Y3  \
 collaborative finishes                     -1.252763 -2.397895       NaN   
 epistemic copulas                          -1.252763 -2.397895       NaN   
 epistemic modals                           -1.252763 -1.704748 -1.609438   
 negotiation of meaning                     -1.252763       NaN       NaN   
 impersonal subject + non-factive verb + NP -1.252763 -1.704748 -1.609438   
 routinized resources                       -1.945910       NaN       NaN   
 question-based responses                   -1.945910 -1.704748 -1.609438   
 formulaic responses                        -1.945910 -1.704748 -1.609438   
 non-factive verb phrase structure          -1.945910 -1.704748 -1.609438   
 code-switching for communicative purposes  -1.945910 -2.397895       NaN   
 subordinate clauses                              NaN -1.299283 -1.609438   
 tense choice to indicate interactive aims        NaN -1.704748 -1.609438   

In [21]:
def get_model_C(input_dim,output_dim):
    inputs=tf.keras.layers.Input(shape=(input_dim,))
    dense=tf.keras.layers.Dense(64,activation='relu')(inputs)
    dense=tf.keras.layers.Dense(64,activation='relu')(dense)
    outputs=tf.keras.layers.Dense(5,activation='softmax')(dense)

    model=tf.keras.models.Model(inputs,outputs)
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')
    return model

In [22]:
def get_BP_result_C(train_x_normal,train_c,test_x_normal,test_c,model_name):
    input_dim=train_x_normal.shape[1]
    output_dim=train_c.shape[1]
    
    result_func=get_result
    metrics_list=['acc','pre','rec','f1s']
    
    train_pred_dict,test_pred_dict={},{}
    train_result_dict,test_result_dict={},{}
    for co in train_c.columns:
        train_y_normal=train_c[[co]]-1
        test_y_normal=test_c[[co]]-1
        model_bp=get_model_C(input_dim,output_dim)
        stopearly=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True,monitor='val_loss')
        
        history=model_bp.fit(train_x_normal,train_y_normal,
                         validation_split=0.1,batch_size=16,
                         epochs=100,callbacks=[stopearly],verbose=0)
        
        test_pred_bp=model_bp.predict(test_x_normal)
        train_pred_bp=model_bp.predict(train_x_normal)
        
        train_pred_bp=train_pred_bp.argmax(1)+1
        test_pred_bp=test_pred_bp.argmax(1)+1
        train_pred_dict[co]=train_pred_bp
        test_pred_dict[co]=test_pred_bp
        
        train_result_dict[co]=result_func(train_y_normal+1,train_pred_bp)
        test_result_dict[co]=result_func(test_y_normal+1,test_pred_bp)
        
    train_result=pd.DataFrame(train_result_dict,index=metrics_list)
    test_result=pd.DataFrame(test_result_dict,index=metrics_list)
    
    train_pred_bp=pd.DataFrame(train_pred_dict,index=train_y_normal.index)
    test_pred_bp=pd.DataFrame(test_pred_dict,index=test_y_normal.index)
    train_true_bp=train_c.copy()
    test_true_bp=test_c.copy()
    return [train_result,test_result],[test_true_bp,test_pred_bp],[train_true_bp,train_pred_bp],None

In [23]:
model_name='BP_C'
result,test_true_pred,train_true_pred,importance=get_BP_result_C(train_x_normal,train_c,
                                                               test_x_normal,test_c,model_name)
result

5/5 [==============================] - 0s 996us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 997us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 755us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[           Y1        Y2        Y3   Y4
 acc  0.700000  0.985714  0.935714  1.0
 pre  0.490000  0.985984  0.875561  1.0
 rec  0.700000  0.985714  0.935714  1.0
 f1s  0.576471  0.985429  0.904639  1.0,
            Y1        Y2        Y3        Y4
 acc  0.685714  0.600000  0.914286  0.942857
 pre  0.470204  0.578571  0.835918  0.915126
 rec  0.685714  0.600000  0.914286  0.942857
 f1s  0.557869  0.589091  0.873348  0.928785]

In [40]:
#=============整理topk重要度的集合===============
type_df=collections.defaultdict(dict)
for model_n,df in task_2_importanc.items():
    for type_ in df.columns:
        type_df[type_][model_n]=df[type_]
        
for type_,df in type_df.items():
    df=pd.DataFrame(df)
    save_csv(df,f'task_2_{type_}_importance_df')
    
#===========预测结果与真实结果的t-test检验==============
def get_t_test_result(task_2_train_true_pred):
    from scipy import stats
    import scipy.stats
    t_test_result={}
    for model_n,true_pred in task_2_train_true_pred.items():
        true,pred=true_pred
        type_t={}
        for type_ in true.columns:
            true_t=true[type_].values
            pred_t=pred[type_].values
            t, pval = scipy.stats.ttest_ind(true_t, pred_t)
            type_t[type_]=pval
        t_test_result[model_n]=type_t
    t_test_result=pd.DataFrame(t_test_result)
    return t_test_result

t_test_result=get_t_test_result(task_2_test_true_pred)
t_train_result=get_t_test_result(task_2_train_true_pred)
save_csv(t_test_result,'t_test_result_test')
save_csv(t_train_result,'t_test_result_train')

# Task3：替换x，只用Token

In [24]:
train_x_normal_=train_x_normal[list(Token_level_label_count.keys())]
test_x_normal_=test_x_normal[list(Token_level_label_count.keys())]

In [25]:
#================Linear===========
model_name='Linear'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result

(                                                 Y1        Y2        Y3  \
 reference word                            -0.030723 -0.083305 -0.028687   
 code-switching for communicative purposes  0.042041 -0.094729 -0.022490   
 subordinate clauses                        0.086206  0.057171  0.018881   
 tense choice to indicate interactive aims  0.107389       NaN       NaN   
 noun & verb collocation in proper form     0.107601       NaN -0.018211   
 negotiation of meaning                          NaN -0.074890  0.026343   
 routinized resources                            NaN -0.151891       NaN   
 
                                                  Y4  
 reference word                             0.030979  
 code-switching for communicative purposes       NaN  
 subordinate clauses                        0.003731  
 tense choice to indicate interactive aims       NaN  
 noun & verb collocation in proper form     0.003676  
 negotiation of meaning                     0.004069  
 rout

In [26]:
#=================RF===============
model_name='RF_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result[1]

(                                                 Y1        Y2        Y3  \
 reference word                             0.124345       NaN       NaN   
 subordinate clauses                        0.154470  0.144168  0.148995   
 negotiation of meaning                     0.162718  0.218265  0.230531   
 tense choice to indicate interactive aims  0.174373  0.191246  0.101673   
 noun & verb collocation in proper form     0.288498  0.167940  0.163210   
 routinized resources                            NaN  0.127410  0.232595   
 code-switching for communicative purposes       NaN       NaN       NaN   
 
                                                  Y4  
 reference word                             0.143946  
 subordinate clauses                        0.206441  
 negotiation of meaning                          NaN  
 tense choice to indicate interactive aims       NaN  
 noun & verb collocation in proper form     0.106525  
 routinized resources                       0.282916  
 code

In [27]:
#=================RF===============
model_name='NB_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result[1]

(                                                 Y1        Y2        Y3  \
 routinized resources                      -0.003556 -0.069021  0.000443   
 code-switching for communicative purposes  0.008628 -0.038098 -0.000497   
 noun & verb collocation in proper form     0.023275       NaN       NaN   
 subordinate clauses                        0.023558  0.024982       NaN   
 tense choice to indicate interactive aims  0.025229       NaN -0.000642   
 reference word                                  NaN -0.029485 -0.000968   
 negotiation of meaning                          NaN -0.035620  0.000809   
 
                                                  Y4  
 routinized resources                      -0.003940  
 code-switching for communicative purposes -0.001129  
 noun & verb collocation in proper form     0.001923  
 subordinate clauses                        0.001204  
 tense choice to indicate interactive aims       NaN  
 reference word                             0.008666  
 nego

In [28]:
model_name='BP_R'
result,test_true_pred,train_true_pred,importance=get_BP_result(train_x_normal_,train_y,
                                                               test_x_normal_,test_y,model_name)
result

5/5 [==============================] - 0s 747us/step


[            Y1        Y2        Y3        Y4
 mae   0.405819  0.336431  0.133853  0.074406
 mse   0.402158  0.331834  0.066440  0.015901
 mape  0.152202  0.140766  0.040352  0.026546
 r2    0.208810  0.271264  0.305722  0.163428,
             Y1        Y2        Y3        Y4
 mae   0.411261  0.310570  0.176376  0.156214
 mse   0.368698  0.217433  0.080675  0.077958
 mape  0.126492  0.119340  0.060089  0.061850
 r2    0.079635 -0.620412 -4.240009 -0.558627]

In [29]:
model_name='Logit'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set

(                                                 Y1        Y2        Y3  \
 tense choice to indicate interactive aims -0.172641 -0.662408       NaN   
 code-switching for communicative purposes -0.243042 -0.346816       NaN   
 routinized resources                       0.291146  0.731935  0.436286   
 negotiation of meaning                    -0.533332  0.378248  0.584746   
 subordinate clauses                       -0.647316 -0.099278 -0.503111   
 noun & verb collocation in proper form          NaN       NaN -0.407147   
 reference word                                  NaN       NaN -0.540526   
 
                                                  Y4  
 tense choice to indicate interactive aims -0.319712  
 code-switching for communicative purposes       NaN  
 routinized resources                       0.768454  
 negotiation of meaning                     0.660891  
 subordinate clauses                       -0.610205  
 noun & verb collocation in proper form    -0.490494  
 refe

In [30]:
model_name='RF_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set

(                                                 Y1        Y2        Y3  \
 negotiation of meaning                     0.139387  0.177733  0.150181   
 subordinate clauses                        0.150979  0.165157  0.152326   
 reference word                             0.152197  0.144017  0.169315   
 tense choice to indicate interactive aims  0.163954  0.183714       NaN   
 noun & verb collocation in proper form     0.267745  0.182251  0.106198   
 routinized resources                            NaN       NaN  0.273343   
 
                                                  Y4  
 negotiation of meaning                     0.131563  
 subordinate clauses                        0.186957  
 reference word                             0.239058  
 tense choice to indicate interactive aims       NaN  
 noun & verb collocation in proper form     0.133323  
 routinized resources                       0.135873  ,
 [           Y1        Y2        Y3        Y4
  acc  0.914286  0.914286  0.97142

In [31]:
model_name='NB_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` para

(                                                 Y1        Y2        Y3  \
 tense choice to indicate interactive aims -1.098612 -1.609438 -1.386294   
 subordinate clauses                       -1.504077 -0.916291 -1.386294   
 routinized resources                      -1.504077 -1.203973       NaN   
 negotiation of meaning                    -1.504077 -0.916291       NaN   
 code-switching for communicative purposes -2.197225 -2.302585 -0.693147   
 reference word                                  NaN       NaN -1.386294   
 noun & verb collocation in proper form          NaN       NaN -1.386294   
 
                                                  Y4  
 tense choice to indicate interactive aims -1.386294  
 subordinate clauses                       -1.386294  
 routinized resources                            NaN  
 negotiation of meaning                          NaN  
 code-switching for communicative purposes -0.693147  
 reference word                            -1.386294  
 noun

In [32]:
model_name='BP_C'
result,test_true_pred,train_true_pred,importance=get_BP_result_C(train_x_normal_,train_c,
                                                               test_x_normal_,test_c,model_name)
result

5/5 [==============================] - 0s 748us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 747us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 747us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 997us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[           Y1        Y2        Y3        Y4
 acc  0.700000  0.800000  0.964286  0.985714
 pre  0.490000  0.788559  0.929884  0.985926
 rec  0.700000  0.800000  0.964286  0.985714
 f1s  0.576471  0.742455  0.946763  0.982196,
            Y1        Y2        Y3        Y4
 acc  0.685714  0.771429  0.914286  0.942857
 pre  0.470204  0.636310  0.835918  0.888980
 rec  0.685714  0.771429  0.914286  0.942857
 f1s  0.557869  0.694189  0.873348  0.915126]

# task4：改变x，使用Utrance

In [33]:
train_x_normal_=train_x_normal[list(Utterance_level_label_count.keys())]
test_x_normal_=test_x_normal[list(Utterance_level_label_count.keys())]

In [34]:
#================Linear===========
model_name='Linear'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result

(                                                  Y1        Y2        Y3  \
 question-based responses                    0.050053       NaN       NaN   
 collaborative finishes                     -0.052884 -0.057033  0.033767   
 backchannels                               -0.063941       NaN       NaN   
 impersonal subject + non-factive verb + NP  0.079380  0.046728  0.054628   
 adjectives/ adverbs expressing possibility  0.118065 -0.074992  0.032900   
 non-factive verb phrase structure                NaN  0.052985  0.049231   
 epistemic modals                                 NaN  0.138277  0.071861   
 feedback in the next turn                        NaN       NaN       NaN   
 
                                                   Y4  
 question-based responses                    0.017190  
 collaborative finishes                     -0.054939  
 backchannels                               -0.004377  
 impersonal subject + non-factive verb + NP       NaN  
 adjectives/ adverbs expr

In [35]:
#=================RF===============
model_name='RF_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result[1]

(                                                  Y1        Y2        Y3  \
 question-based responses                    0.114726  0.130049       NaN   
 impersonal subject + non-factive verb + NP  0.147039       NaN  0.152032   
 adjectives/ adverbs expressing possibility  0.157818  0.208605  0.114796   
 feedback in the next turn                   0.164283  0.162848  0.112791   
 backchannels                                0.176179  0.141270       NaN   
 formulaic responses                              NaN  0.132936       NaN   
 epistemic modals                                 NaN       NaN  0.098426   
 collaborative finishes                           NaN       NaN  0.209903   
 epistemic copulas                                NaN       NaN       NaN   
 
                                                   Y4  
 question-based responses                         NaN  
 impersonal subject + non-factive verb + NP       NaN  
 adjectives/ adverbs expressing possibility  0.091282  
 fee

In [36]:
#=================RF===============
model_name='NB_R'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_y,
                                                                test_x_normal_,test_y,
                                                                model_name)
importance,result[1]

(                                                  Y1        Y2        Y3  \
 epistemic modals                            0.004542  0.034573  0.048915   
 collaborative finishes                     -0.004610 -0.016063  0.017788   
 backchannels                               -0.006409       NaN       NaN   
 impersonal subject + non-factive verb + NP  0.007008  0.019094  0.042254   
 adjectives/ adverbs expressing possibility  0.010543 -0.013549  0.022696   
 non-factive verb phrase structure                NaN  0.017240  0.035378   
 formulaic responses                              NaN       NaN       NaN   
 question-based responses                         NaN       NaN       NaN   
 
                                                   Y4  
 epistemic modals                                 NaN  
 collaborative finishes                     -0.031586  
 backchannels                               -0.004104  
 impersonal subject + non-factive verb + NP       NaN  
 adjectives/ adverbs expr

In [37]:
model_name='BP_R'
result,test_true_pred,train_true_pred,importance=get_BP_result(train_x_normal_,train_y,
                                                               test_x_normal_,test_y,model_name)
result

5/5 [==============================] - 0s 996us/step


[            Y1        Y2        Y3        Y4
 mae   0.344749  0.319934  0.109670  0.059843
 mse   0.408031  0.336264  0.066196  0.014159
 mape  0.129870  0.142312  0.033402  0.021488
 r2    0.197256  0.261537  0.308270  0.255111,
             Y1        Y2        Y3        Y4
 mae   0.401037  0.262078  0.094068  0.130187
 mse   0.406271  0.216370  0.019657  0.086981
 mape  0.124952  0.102766  0.031972  0.053699
 r2   -0.014156 -0.612489 -0.276776 -0.739030]

In [38]:
model_name='Logit'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set

(                                                  Y1        Y2        Y3  \
 adjectives/ adverbs expressing possibility -0.184910       NaN -0.634980   
 impersonal subject + non-factive verb + NP  0.188682       NaN       NaN   
 epistemic copulas                          -0.248192 -0.275965  0.256244   
 question-based responses                   -0.336025       NaN -0.376318   
 epistemic modals                           -0.448760       NaN -0.223982   
 backchannels                                     NaN  0.284784       NaN   
 non-factive verb phrase structure                NaN  0.320516       NaN   
 collaborative finishes                           NaN -0.447255  0.596214   
 formulaic responses                              NaN -0.496914       NaN   
 
                                                   Y4  
 adjectives/ adverbs expressing possibility -0.566951  
 impersonal subject + non-factive verb + NP       NaN  
 epistemic copulas                           0.228585  
 que

In [39]:
model_name='RF_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(                                                  Y1        Y2        Y3  \
 formulaic responses                         0.115197  0.135452       NaN   
 question-based responses                    0.117645  0.149015       NaN   
 adjectives/ adverbs expressing possibility  0.142520  0.150961       NaN   
 feedback in the next turn                   0.163972  0.154238  0.152521   
 backchannels                                0.169295  0.118339  0.105877   
 epistemic modals                                 NaN       NaN  0.128288   
 collaborative finishes                           NaN       NaN  0.178772   
 epistemic copulas                                NaN       NaN  0.179103   
 
                                                   Y4  
 formulaic responses                              NaN  
 question-based responses                    0.123961  
 adjectives/ adverbs expressing possibility       NaN  
 feedback in the next turn                   0.127207  
 backchannels            

In [40]:
model_name='NB_C'
result,test_true_pred,train_true_pred,importance=get_all_result(train_x_normal_,train_c,
                                                                test_x_normal_,test_c,
                                                                model_name)
importance,result

D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` para

(                                                  Y1        Y2        Y3  \
 impersonal subject + non-factive verb + NP -1.504077 -1.609438 -1.386294   
 question-based responses                   -2.197225       NaN       NaN   
 epistemic modals                           -2.197225       NaN -1.386294   
 non-factive verb phrase structure          -2.197225 -1.609438 -1.386294   
 epistemic copulas                          -2.197225 -2.302585       NaN   
 feedback in the next turn                        NaN -1.609438 -1.386294   
 collaborative finishes                           NaN -2.302585       NaN   
 adjectives/ adverbs expressing possibility       NaN       NaN -1.386294   
 
                                                   Y4  
 impersonal subject + non-factive verb + NP -1.386294  
 question-based responses                         NaN  
 epistemic modals                           -1.386294  
 non-factive verb phrase structure          -1.386294  
 epistemic copulas       

In [41]:
model_name='BP_C'
result,test_true_pred,train_true_pred,importance=get_BP_result_C(train_x_normal_,train_c,
                                                               test_x_normal_,test_c,model_name)
result

5/5 [==============================] - 0s 748us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 997us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 748us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 0s 997us/step


D:\ANACONDA\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[           Y1        Y2        Y3        Y4
 acc  0.700000  0.842857  0.950000  0.971429
 pre  0.490000  0.870295  0.916822  0.961187
 rec  0.700000  0.842857  0.950000  0.971429
 f1s  0.576471  0.813432  0.930412  0.964643,
            Y1        Y2        Y3        Y4
 acc  0.685714  0.714286  0.914286  0.914286
 pre  0.470204  0.602679  0.835918  0.887395
 rec  0.685714  0.714286  0.914286  0.914286
 f1s  0.557869  0.653753  0.873348  0.900640]